![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/automated-machine-learning/regression/auto-ml-regression.png)

In [1]:
!git pull

Already up to date.


In [18]:
import logging

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.datastore import Datastore
from azureml.core.dataset import Dataset
from azureml.data.data_reference import DataReference
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from sklearn.metrics import mean_squared_error, r2_score
print(azureml.core.VERSION) #should be 1.0.57 or greater

1.0.57


In [3]:
ws = Workspace.from_config()
experiment_name = 'house_prices_regression'
project_folder = './sample_projects/house_prices_regression'
experiment = Experiment(ws, experiment_name)

## Data
This uses the AzureML datastore and dataset api's, as well as data manipulation, with keep_ and drop_ columns, and random_split.

In [4]:
datastore_name = 'edvanstorage__azureml'
dataset_path = 'boston_houses/boston_data.csv'
datastore = Datastore.get(ws, datastore_name)
data_reference = DataReference(datastore, data_reference_name="boston_data", path_on_datastore=dataset_path)
boston_ds = Dataset.Tabular.from_delimited_files(data_reference)
boston_ds = boston_ds.drop_columns('Column1')
boston_ds.take(3).to_pandas_dataframe()

,CrimeRate,ResidentialZoning,IndustrialZoning,OnRiver,NOXConcentration,NumberOfRooms,PreWarHouses,DistanceToEmployment,DistanceToHighways,PropertyTaxRate,ParentTeachRatio,ProportionAA,LowerStatusProportion,Price
0,0.01,18.00,2.31,0.00,0.54,6.58,65.20,4.09,1.00,296.00,15.30,396.90,4.98,24.00
1,0.03,0.00,7.07,0.00,0.47,6.42,78.90,4.97,2.00,242.00,17.80,396.90,9.14,21.60
2,0.03,0.00,7.07,0.00,0.47,7.18,61.10,4.97,2.00,242.00,17.80,392.83,4.03,34.70


In [5]:
b_train, b_test = boston_ds.random_split(0.2, seed=84)
X_train = b_train.drop_columns('Price')
y_train = b_train.keep_columns('Price')
X_test = b_test.drop_columns('Price')
y_test = b_test.keep_columns('Price')

## Remote compute
Setup the environments for remote compute, we might not need it...

In [6]:
compute_target = ws.compute_targets['cpucluster']
run_config = RunConfiguration(framework="python")
run_config.target = compute_target
run_config.environment.docker.enabled = True
run_config.environment.docker.base_image = azureml.core.runconfig.DEFAULT_CPU_IMAGE

dependencies = CondaDependencies.create(
    pip_packages=["scikit-learn", "scipy", "numpy"])
run_config.environment.python.conda_dependencies = dependencies

## Train

Instantiate an `AutoMLConfig` object to specify the settings for both local and remote runs and data used to run the experiment.

In [7]:
aml_config = {"task": 'regression',
         "iteration_timeout_minutes": 10,
         "primary_metric": 'normalized_root_mean_squared_error',
         "debug_log": 'automl.log',
         "verbosity": logging.INFO,
         "enable_early_stopping": True,
         "X": X_train, 
         "y": y_train,
         "path": project_folder}

local_run = False
if local_run:
    numb_run_config = {
        'iterations': 10,
        'n_cross_validations': 5
    }
else:
    numb_run_config = {
        'iterations': 250,
        'n_cross_validations': 10
    }    
    remote_config = {
        'compute_target': compute_target,
        'run_configuration': run_config,
        'max_cores_per_iteration': 2,
        'max_concurrent_iterations': 10,
    }
    aml_config.update(remote_config)
aml_config.update(numb_run_config)
    
automl_config = AutoMLConfig(**aml_config)

In [8]:
show_run = False
if 'compute_target' in aml_config:
    print(f"Running {aml_config['iterations']} iterations on remote compute (with {compute_target.vm_size} VM's), with {aml_config['n_cross_validations']}-fold cross validation.")
else:
    print(f"Running {aml_config['iterations']} iterations on local compute, with {aml_config['n_cross_validations']}-fold cross validation.")
if not show_run:
    print("Not showing output here. Please run the Widget cell below.")
else:
    print("---------------------------------------------------------------------------------")
run = experiment.submit(automl_config, show_output = show_run)

Running 250 iterations on remote compute (with STANDARD_D2_V2 VM's), with 10-fold cross validation.
Not showing output here. Please run the Widget cell below.


In [11]:
from azureml.widgets import RunDetails
RunDetails(run).show() 

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [12]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
house_prices_regression,AutoML_f21709b6-891f-454f-b677-f0b6b8bb626f,automl,Completed,Link to Azure Portal,Link to Documentation



#### Retrieve All Child Runs
You can also use SDK methods to fetch all the child runs and see individual metrics that we log.

In [13]:
children = list(run.get_children())
metricslist = {}
for r in children:
    properties = r.get_properties()
    metrics = {k: v for k, v in r.get_metrics().items() if isinstance(v, float)}
    metricslist[int(properties['iteration'])] = metrics

rundata = pd.DataFrame(metricslist).sort_index(1)
rundata

,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
explained_variance,0.65,0.66,0.74,0.78,0.77,0.67,0.56,0.73,0.52,0.60,...,0.78,0.78,0.75,0.67,0.75,0.78,0.79,0.81,0.87,0.86
mean_absolute_error,3.09,3.09,2.57,2.29,2.36,2.84,2.96,2.40,3.30,2.93,...,2.40,2.24,2.53,2.86,2.26,2.27,2.22,2.00,1.68,1.81
mean_absolute_percentage_error,15.53,15.65,13.73,12.07,12.82,15.45,13.56,11.81,16.10,13.79,...,12.74,12.16,14.39,14.68,11.04,11.11,11.40,10.25,8.58,9.20
median_absolute_error,2.45,2.46,1.75,1.83,1.96,2.15,1.74,1.65,2.31,2.10,...,1.93,1.63,1.92,2.01,1.67,1.70,1.78,1.39,1.16,1.37
normalized_mean_absolute_error,0.07,0.07,0.06,0.05,0.05,0.07,0.07,0.06,0.08,0.07,...,0.06,0.05,0.06,0.07,0.05,0.05,0.05,0.05,0.04,0.04
normalized_median_absolute_error,0.06,0.06,0.04,0.04,0.05,0.05,0.04,0.04,0.05,0.05,...,0.04,0.04,0.04,0.05,0.04,0.04,0.04,0.03,0.03,0.03
normalized_root_mean_squared_error,0.09,0.09,0.08,0.07,0.07,0.09,0.10,0.08,0.11,0.09,...,0.08,0.07,0.08,0.09,0.08,0.07,0.07,0.07,0.05,0.06
normalized_root_mean_squared_log_error,0.10,0.10,0.10,0.08,0.08,0.13,0.10,0.08,0.10,0.09,...,0.08,0.08,0.09,0.10,0.08,0.07,0.07,0.07,0.06,0.06
r2_score,0.58,0.58,0.71,0.75,0.74,0.64,0.52,0.69,0.42,0.56,...,0.74,0.76,0.71,0.62,0.73,0.76,0.76,0.79,0.86,0.84
root_mean_squared_error,3.98,3.97,3.37,3.12,3.19,3.71,4.30,3.35,4.52,3.91,...,3.25,3.10,3.44,3.92,3.25,3.09,3.09,2.86,2.34,2.45


### Retrieve the Best Model

Below we select the best pipeline from our iterations. The `get_output` method returns the best run and the fitted model. The Model includes the pipeline and any pre-processing.  Overloads on `get_output` allow you to retrieve the best run and fitted model for *any* logged metric or for a particular *iteration*.

In [14]:
best_run, fitted_model = run.get_output()
print(best_run)
print("----------------------------")
print(fitted_model)

ModuleNotFoundError: No module named 'sklearn.ensemble._gb_losses'

#### Best Model Based on Any Other Metric
Show the run and the model that has the smallest `root_mean_squared_error` value (which turned out to be the same as the one with largest `spearman_correlation` value):

In [ ]:
lookup_metric = "spearman_correlation"
best_run, fitted_model = run.get_output(metric = lookup_metric)
print(best_run)
print("----------------------------")
print(fitted_model)

#### Model from a Specific Iteration
Show the run and the model from the third iteration:

In [ ]:
iteration = 3
third_run, third_model = run.get_output(iteration = iteration)
print(third_run)
print("----------------------------")
print(third_model)

## Test

Predict on training and test set, and calculate residual values.

In [16]:
X_train_pd = X_train.to_pandas_dataframe()
y_train_pd = y_train.to_pandas_dataframe().squeeze()
X_test_pd = X_test.to_pandas_dataframe()
y_test_pd = y_test.to_pandas_dataframe().squeeze()

y_pred_train = fitted_model.predict(X_train_pd)
y_residual_train = y_train_pd - y_pred_train

y_pred_test = fitted_model.predict(X_test_pd)
y_residual_test = y_test_pd - y_pred_test

In [ ]:
%matplotlib inline

# Set up a multi-plot chart.
f, (a0, a1) = plt.subplots(1, 2, gridspec_kw = {'width_ratios':[1, 1], 'wspace':0, 'hspace': 0})
f.suptitle('Regression Residual Values', fontsize = 18)
f.set_figheight(6)
f.set_figwidth(16)

# Plot residual values of training set.
a0.axis([0, 360, -200, 200])
a0.plot(y_residual_train, 'bo', alpha = 0.5)
a0.plot([-10,360],[0,0], 'r-', lw = 3)
a0.text(16,170,'RMSE = {0:.2f}'.format(np.sqrt(mean_squared_error(y_train_pd, y_pred_train))), fontsize = 12)
a0.text(16,140,'R2 score = {0:.2f}'.format(r2_score(y_train_pd, y_pred_train)), fontsize = 12)
a0.set_xlabel('Training samples', fontsize = 12)
a0.set_ylabel('Residual Values', fontsize = 12)

# Plot a histogram.
a0.hist(y_residual_train, orientation = 'horizontal', color = 'b', bins = 10, histtype = 'step')
a0.hist(y_residual_train, orientation = 'horizontal', color = 'b', alpha = 0.2, bins = 10)

# Plot residual values of test set.
a1.axis([0, 90, -200, 200])
a1.plot(y_residual_test, 'bo', alpha = 0.5)
a1.plot([-10,360],[0,0], 'r-', lw = 3)
a1.text(5,170,'RMSE = {0:.2f}'.format(np.sqrt(mean_squared_error(y_test_pd, y_pred_test))), fontsize = 12)
a1.text(5,140,'R2 score = {0:.2f}'.format(r2_score(y_test_pd, y_pred_test)), fontsize = 12)
a1.set_xlabel('Test samples', fontsize = 12)
a1.set_yticklabels([])

# Plot a histogram.
a1.hist(y_residual_test, orientation = 'horizontal', color = 'b', bins = 10, histtype = 'step')
a1.hist(y_residual_test, orientation = 'horizontal', color = 'b', alpha = 0.2, bins = 10)

plt.show()

In [21]:
all_rmse = {}
for ite in range(aml_config['iterations']-2):
    try:
        i_run, model = run.get_output(iteration = ite)
        y_pred_test = model.predict(X_test_pd)
        all_rmse[ite] = np.sqrt(mean_squared_error(y_test_pd, y_pred_test))
    except Exception as e:
        print(f"Iteration {ite} gave an error {e}.")

Iteration 47 gave an error No module named 'sklearn.ensemble._gb_losses'.
Iteration 52 gave an error No module named 'sklearn.ensemble._gb_losses'.
Iteration 56 gave an error No module named 'sklearn.ensemble._gb_losses'.
Iteration 60 gave an error No module named 'sklearn.ensemble._gb_losses'.
Iteration 64 gave an error No module named 'sklearn.ensemble._gb_losses'.
Iteration 67 gave an error InvalidRunState:
	Message: Run 67 is in Canceled state.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "inner_error": {
            "code": "ValidationError",
            "inner_error": {
                "code": "ScenarioNotSuported",
                "inner_error": {
                    "code": "InvalidRunState"
                }
            }
        },
        "message": "Run 67 is in Canceled state."
    }
}.
Iteration 69 gave an error InvalidRunState:
	Message: Run 69 is in Canceled state.
	InnerException None
	ErrorResponse 
{
    "error": {
      

In [22]:
all_rmse

{0: 6.064030259958559,
 1: 5.984815647740434,
 2: 5.461779759024959,
 3: 5.986299431153826,
 4: 5.756417944463689,
 5: 5.318597708122248,
 6: 6.380587988052985,
 7: 5.455929821099788,
 8: 6.544755899744011,
 9: 6.079802051156183,
 10: 6.150815356587078,
 11: 5.841920245201934,
 12: 5.669702511277894,
 13: 5.481638878118461,
 14: 5.452174955547824,
 15: 5.425388885253178,
 16: 6.055743733212375,
 17: 5.805117388913986,
 18: 6.353246789317016,
 19: 5.550176300178423,
 20: 5.91608586571899,
 21: 5.6791860424359415,
 22: 6.320560979298308,
 23: 7.144471651375276,
 24: 6.320550743576675,
 25: 5.7381528284694605,
 26: 5.841393729909538,
 27: 5.657233842349258,
 28: 5.835952875699271,
 29: 5.347543877983778,
 30: 5.7006994939940805,
 31: 5.719850202705487,
 32: 5.308106857566783,
 33: 5.469461333282142,
 34: 6.045880316997678,
 35: 5.152186397190498,
 36: 5.0301957004044695,
 37: 5.039365293877263,
 38: 5.585520878811329,
 39: 5.237685965147543,
 40: 5.964005616049966,
 41: 5.545046184793176,

In [66]:
best_iteration = sorted(all_rmse, key=all_rmse.get, reverse=False)[0]
print(f'Best iteration is number {best_iteration}')
print("--------------")
best_test_run, best_test_model = run.get_output(iteration = best_iteration)
print(best_test_run)
print(best_test_run.get_file_names())
print("--------------")
print(best_test_model)
# print(best_test_model.get_model_path())

Best iteration is number 141
--------------


AttributeError: 'function' object has no attribute 'get_output'

In [24]:
dependencies = run.get_run_sdk_dependencies(iteration = best_iteration)

WARNING - Below packages were used for model training but missing in current environment:
WARNING - Package:azureml-defaults, training version:1.0.57
Package:azureml-explain-model, training version:1.0.57
Package:azureml-model-management-sdk, training version:1.0.1b6.post1


In [108]:
for p in ['azureml-train-automl', 'azureml-core', 'azureml-defaults']:
    print('{}\t{}'.format(p, dependencies[p]))

azureml-train-automl	1.0.57
azureml-core	1.0.57.1
azureml-defaults	1.0.57


In [111]:
myenv = CondaDependencies.create(conda_packages=['numpy','scikit-learn','py-xgboost==0.80'], pip_packages=['azureml-train-automl','azureml-core', 'azureml-defaults'])

conda_env_file_name = 'conda_dependencies.yml'
myenv.save_to_file('.', conda_env_file_name)

'conda_dependencies.yml'

## Register model and commit dependencies and score files first

Since the Release is triggered from a new version of the model, first the dependent files are updated in the repo, then the new model is registered, which triggers the release.

In [113]:
dep_changes = !git diff conda_dependencies.yml
score_changes = !git diff score.py
inference_changes = !git diff inference_config.json
aci_config_changes = !git diff aci_deployment_config.json
aks_config_changes = !git diff aks_deployment_config.json
if len(dep_changes) > 0:
    !git add conda_dependencies.yml
if len(score_changes) > 0:
    !git add score.py
if len(inference_changes) > 0:
    !git add inference_config.json
if len(aci_config_changes) > 0:
    !git add aci_deployment_config.json
if len(aks_config_changes) > 0:
    !git add aks_deployment_config.json
if len(dep_changes)+len(score_changes)+len(inference_changes)+len(aci_config_changes)+len(aks_config_changes)>0:
    print("Changes to commit. Committing and pushing now")
    !git commit -m "Commit from NB"
    !git push
else:
    print("No changes to commit.")

Changes to commit. Committing and pushing now
[master ff72e13] Commit from NB
 1 file changed, 21 insertions(+), 26 deletions(-)


To https://github.com/eavanvalkenburg/azureml_demo.git
   f8df0c4..ff72e13  master -> master


In [114]:
name = "house_prices_regression"
deploy = True

# best_test_model.register(ws, )
model = best_test_run.register_model(name, model_path='outputs/model.pkl')
print(model.name, model.id, model.version, sep='\t')

house_prices_regression	house_prices_regression:7	7


In [99]:

import pickle
import json
import numpy
from azureml.core.model import Model

def init():
    global model
    from sklearn.externals import joblib

    # load the model from file into a global object
    model_path = Model.get_model_path(model_name="house_prices_regression")
    model = joblib.load(model_path)


# note you can pass in multiple rows for scoring
def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        data = numpy.array(data)
        result = model.predict(data)
        # you can return any datatype as long as it is JSON-serializable
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error


In [100]:
test_data = {"data": [[ 0.00632, 18.0, 2.31, 0.0,0.538, 6.575, 65.2, 4.09, 1.0, 296.0, 15.3, 396.9, 4.98 ]]}


In [101]:

run(json.dumps(test_data))

"'Model' object has no attribute 'predict'"

In [65]:
from sklearn.externals import joblib
# model_path = Model.get_model_path(model_name="sample_projects/house_prices_regression")
joblib.load(model)

TypeError: expected str, bytes or os.PathLike object, not Model

In [112]:
from azureml.core.webservice import LocalWebservice

from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.environment import Environment

environment = Environment("LocalDeploy")
environment.python.conda_dependencies = CondaDependencies("conda_dependencies.yml")

from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script="score.py", environment=environment)

deployment_config = LocalWebservice.deploy_configuration(port=6789)

local_service = Model.deploy(ws, "test", [model], inference_config, deployment_config)

local_service.wait_for_deployment()

ERROR - Failed to create Docker client. Is Docker running/installed?
When you deploy locally, we download a dockerfile
execute docker build on it, and docker run the built container for you
Error: Error while fetching server API version: (2, 'WaitNamedPipe', 'The system cannot find the file specified.')



WebserviceException: WebserviceException:
	Message: Failed to create Docker client. Is Docker running/installed?
When you deploy locally, we download a dockerfile
execute docker build on it, and docker run the built container for you
Error: Error while fetching server API version: (2, 'WaitNamedPipe', 'The system cannot find the file specified.')
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Failed to create Docker client. Is Docker running/installed?\nWhen you deploy locally, we download a dockerfile\nexecute docker build on it, and docker run the built container for you\nError: Error while fetching server API version: (2, 'WaitNamedPipe', 'The system cannot find the file specified.')"
    }
}